In [1]:
# Load packages
library(mgcv)
library(ggplot2)
library(tidyverse)
library(lubridate)
library(gridExtra)
library(grid)
library(writexl)
library(magrittr)
library(rstatix)
library(MASS)

Loading required package: nlme

This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v tibble  3.1.7     v dplyr   1.0.9
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1
v purrr   0.3.4     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::collapse() masks nlme::collapse()
x dplyr::filter()   masks stats::filter()
x dplyr::lag()      masks stats::lag()


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union



Attaching package: 'gridExtra'


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attac

In [79]:
PATH <- file.path("C:/Users/julia/OneDrive - London School of Hygiene and Tropical Medicine/Documents/LSHTM/Project") #"/home/andewegs/1_RSV_scripts/"
setwd(PATH)
getwd()
rsv.data <- read_csv(file = "Data/infection_status.csv")

[1] "C:/Users/julia/OneDrive - London School of Hygiene and Tropical Medicine/Documents/LSHTM/Project"

Rows: 682 Columns: 23
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
dbl  (20): age_days, infection, IgG_PreF, IgA_PreF, IgG_PostF, IgA_PostF, Ig...
lgl   (1): IgA_Nn_household
date  (2): birthday, consultdate

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [80]:
spring <- c(3, 4, 5)
summer <- c(6, 7, 8)
autumn <- c (9, 10, 11)
winter <- c(1, 2, 12)

rsv.data <- rsv.data %>%
 mutate(
    Birth_mo = birthday %>% month()
  )%>%
    mutate(
        season_birth = case_when(
            Birth_mo %in% spring ~ "Spring",
            Birth_mo %in% summer ~ "Summer",
            Birth_mo %in% autumn ~ "Autumn",
            Birth_mo %in% winter ~ "Winter")
        )
head(rsv.data)

age_days,birthday,consultdate,infection,IgG_PreF,IgA_PreF,IgG_PostF,IgA_PostF,IgG_Ga,IgA_Ga,⋯,household59,visitnursery_child,visitnursery_house,pregnancytime,contacttotal,contact04,contact59,sex,Birth_mo,season_birth
<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
676,2004-04-01,2006-02-06,1,0.3565,0.60,9.3264,0.08,0.260000,0.01,⋯,0,0,0,NA,3,1,0,2,4,Spring
600,2004-07-23,2006-03-15,1,772.3431,33.15,473.5323,14.52,3.130699,0.02,⋯,0,1,0,NA,29,20,0,1,7,Summer
588,2004-08-09,2006-03-20,0,0.1800,0.03,0.2100,0.19,0.580000,0.03,⋯,0,0,0,NA,4,1,0,2,8,Summer
591,2004-08-06,2006-03-20,1,740.2907,70.38,466.2055,49.78,1.886910,0.05,⋯,1,0,0,NA,2,0,0,2,8,Summer
1184,2002-12-29,2006-03-27,1,1999.2353,NA,1459.5921,NA,55.896003,NA,⋯,1,0,0,NA,6,2,3,1,12,Winter
686,2004-05-12,2006-03-29,1,807.2200,NA,838.7400,NA,15.000000,NA,⋯,0,0,0,NA,2,0,0,2,5,Spring


In [81]:
rsv.data$agegrp <- cut(rsv.data$age_days,
                    breaks=c(seq(0,730, by=30.25*2),
                             seq(909,2000, by=30.25*6)), 
                    include.lowest = T, right=F)

head(rsv.data)

age_days,birthday,consultdate,infection,IgG_PreF,IgA_PreF,IgG_PostF,IgA_PostF,IgG_Ga,IgA_Ga,⋯,visitnursery_child,visitnursery_house,pregnancytime,contacttotal,contact04,contact59,sex,Birth_mo,season_birth,agegrp
<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
676,2004-04-01,2006-02-06,1,0.3565,0.60,9.3264,0.08,0.260000,0.01,⋯,0,0,NA,3,1,0,2,4,Spring,"[666,726)"
600,2004-07-23,2006-03-15,1,772.3431,33.15,473.5323,14.52,3.130699,0.02,⋯,1,0,NA,29,20,0,1,7,Summer,"[544,605)"
588,2004-08-09,2006-03-20,0,0.1800,0.03,0.2100,0.19,0.580000,0.03,⋯,0,0,NA,4,1,0,2,8,Summer,"[544,605)"
591,2004-08-06,2006-03-20,1,740.2907,70.38,466.2055,49.78,1.886910,0.05,⋯,0,0,NA,2,0,0,2,8,Summer,"[544,605)"
1184,2002-12-29,2006-03-27,1,1999.2353,NA,1459.5921,NA,55.896003,NA,⋯,0,0,NA,6,2,3,1,12,Winter,"[1.09e+03,1.27e+03)"
686,2004-05-12,2006-03-29,1,807.2200,NA,838.7400,NA,15.000000,NA,⋯,0,0,NA,2,0,0,2,5,Spring,"[666,726)"


In [82]:
rsv.data <- subset (rsv.data, !is.na(visitnursery_child))
daycare <- subset(rsv.data, visitnursery_child == 1)
no_day <- subset(rsv.data, visitnursery_child == 0)

In [83]:
head(daycare)

age_days,birthday,consultdate,infection,IgG_PreF,IgA_PreF,IgG_PostF,IgA_PostF,IgG_Ga,IgA_Ga,⋯,visitnursery_child,visitnursery_house,pregnancytime,contacttotal,contact04,contact59,sex,Birth_mo,season_birth,agegrp
<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
600,2004-07-23,2006-03-15,1,772.34314,33.15,473.5323,14.52,3.130699,0.02,⋯,1,0,NA,29,20,0,1,7,Summer,"[544,605)"
1282,2002-09-24,2006-03-29,1,2222.81750,NA,1500.0000,NA,19.455700,NA,⋯,1,0,NA,NA,NA,NA,2,9,Autumn,"[1.27e+03,1.45e+03)"
976,2003-08-01,2006-04-03,1,0.53000,NA,1.2300,NA,2.250200,NA,⋯,1,0,NA,4,0,0,2,8,Summer,"[909,1.09e+03)"
1268,2002-10-13,2006-04-03,1,1357.84000,NA,5000.0000,NA,2.883200,NA,⋯,1,0,NA,10,2,0,2,10,Autumn,"[1.09e+03,1.27e+03)"
957,2003-08-22,2006-04-05,1,561.33960,NA,1058.4328,NA,53.346847,NA,⋯,1,0,NA,6,1,1,1,8,Summer,"[909,1.09e+03)"
567,2004-09-20,2006-04-10,1,22.06873,0.26,109.5198,1.07,2.590000,0.02,⋯,1,0,NA,5,3,0,1,9,Autumn,"[544,605)"


In [144]:
one <- subset(daycare, agegrp == "[1.82e+03,2e+03]")
two <- subset(no_day, agegrp == "[1.82e+03,2e+03]")

In [90]:
one

age_days,birthday,consultdate,infection,IgG_PreF,IgA_PreF,IgG_PostF,IgA_PostF,IgG_Ga,IgA_Ga,⋯,visitnursery_child,visitnursery_house,pregnancytime,contacttotal,contact04,contact59,sex,Birth_mo,season_birth,agegrp
<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
82,2006-08-04,2006-10-25,0,29.27499,0.03,28.92621,0.28,0.830000,0.02,⋯,1,0,NA,NA,NA,NA,1,8,Summer,"[60.5,121)"
102,2006-11-16,2007-02-26,1,73.48835,0.11,229.07401,0.90,3.599333,0.03,⋯,1,1,NA,3,1,0,1,11,Autumn,"[60.5,121)"
107,2007-01-21,2007-05-08,0,62.60582,0.01,70.56261,0.01,2.680280,0.02,⋯,1,0,NA,3,0,0,1,1,Winter,"[60.5,121)"
81,2007-03-10,2007-05-30,0,88.51545,0.02,147.60778,0.02,0.802134,0.02,⋯,1,1,NA,NA,NA,NA,1,3,Spring,"[60.5,121)"
104,2007-02-27,2007-06-11,0,197.44000,0.02,226.76000,0.03,0.920000,0.02,⋯,1,0,NA,NA,NA,NA,1,2,Winter,"[60.5,121)"
96,2016-03-11,2016-06-15,0,113.03000,0.02,65.41000,0.02,1.410000,0.02,⋯,1,1,39,17,10,0,2,3,Spring,"[60.5,121)"
110,2016-03-10,2016-06-28,0,129.79000,0.02,52.74000,0.02,18.210000,0.03,⋯,1,0,NA,0,NA,NA,1,3,Spring,"[60.5,121)"
100,2016-03-22,2016-06-30,0,59.11000,0.02,38.30000,0.02,0.710000,0.02,⋯,1,1,41,3,0,0,2,3,Spring,"[60.5,121)"
114,2016-05-17,2016-09-08,0,52.30000,0.02,82.80000,0.02,0.620000,0.02,⋯,1,0,41,17,14,0,1,5,Spring,"[60.5,121)"


In [91]:
two

age_days,birthday,consultdate,infection,IgG_PreF,IgA_PreF,IgG_PostF,IgA_PostF,IgG_Ga,IgA_Ga,⋯,visitnursery_child,visitnursery_house,pregnancytime,contacttotal,contact04,contact59,sex,Birth_mo,season_birth,agegrp
<dbl>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
66,2006-01-27,2006-04-03,0,117.49773,0.02,31.020256,0.02,3.7500000,0.02,⋯,0,0,NA,NA,NA,NA,1,1,Winter,"[60.5,121)"
70,2006-01-31,2006-04-11,0,83.92986,0.02,48.306774,0.02,1.1800000,0.01,⋯,0,1,NA,5,1,0,2,1,Winter,"[60.5,121)"
97,2006-03-16,2006-06-21,1,117.75099,0.05,52.652352,1.56,4.8572239,0.07,⋯,0,0,NA,NA,NA,NA,1,3,Spring,"[60.5,121)"
85,2006-06-18,2006-09-11,0,70.03440,0.02,57.779319,0.04,1.5491327,0.02,⋯,0,1,NA,NA,NA,NA,1,6,Summer,"[60.5,121)"
85,2006-06-27,2006-09-20,0,116.75605,0.02,76.090000,0.02,0.9700000,0.02,⋯,0,1,NA,4,1,0,2,6,Summer,"[60.5,121)"
68,2006-07-28,2006-10-04,0,87.06760,0.02,55.634513,0.03,1.9393241,0.02,⋯,0,0,NA,NA,NA,NA,1,7,Summer,"[60.5,121)"
108,2006-06-18,2006-10-04,0,165.23688,0.10,72.485884,0.03,1.3308644,0.02,⋯,0,0,NA,NA,NA,NA,1,6,Summer,"[60.5,121)"
67,2006-08-10,2006-10-16,0,291.37000,0.02,341.850000,0.05,2.7600000,0.02,⋯,0,0,NA,NA,NA,NA,2,8,Summer,"[60.5,121)"
120,2006-06-18,2006-10-16,0,1.02000,0.02,1.500000,0.02,0.1100000,0.02,⋯,0,0,NA,12,3,2,1,6,Summer,"[60.5,121)"


In [145]:
three <- rbind(one, two)
#head(three)

In [146]:
car.data = table(three$visitnursery_child, three$infection) 
print(car.data)

# Perform the Chi-Square test.
print(chisq.test(car.data))

   
    1
  0 5
  1 1


Warning message in chisq.test(car.data):
"Chi-squared approximation may be incorrect"



	Chi-squared test for given probabilities

data:  car.data
X-squared = 2.6667, df = 1, p-value = 0.1025

